## 영화 5개 입력받기

In [ ]:
print("좋아하는 영화 5개를 입력해주세요(예: 명량 2014, 기생충 2019, ...): ")
movie_list = input().split(", ")
print(movie_list)

좋아하는 영화 5개를 입력해주세요(예: 명량 2014, 기생충 2019, ...): 
모가디슈 2021, 천문 2019, 라라랜드 2016, 크루엘라 2021, 블랙 위도우 2021
['모가디슈 2021', '천문 2019', '라라랜드 2016', '크루엘라 2021', '블랙 위도우 2021']


## 영화별 출연진 정보 가져오기

영화 목록에서 영화코드 추출

In [ ]:
import requests
import json

In [ ]:
movie_cd_list = []
director_list = []
selected_movie_list = []
yourcode = # 영화진흥원 api 키를 입력하세요
for movie in movie_list:
  movie_cd_url = 'https://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieList.json?key='+yourcode+'&movieNm='+movie[:-5]
  res = requests.get(movie_cd_url)
  text = res.text
  d = json.loads(text)
  if d['movieListResult']['totCnt'] > 1:    # 동명의 영화가 있을 경우
    item_year = movie[-4:]   # 개봉연도로 구분
    for item in d['movieListResult']['movieList']:
      if item_year == item['openDt'][:4]:
          # director_list = director['peopleNm']
        movie_cd_list.append([item['movieCd'],
                              item['movieNm'],
                              item['openDt'][:4]])
        selected_movie_list.append(item['movieNm'])
  if d['movieListResult']['totCnt'] == 1:
    for item in d['movieListResult']['movieList']:
      movie_cd_list.append([item['movieCd'],
                            item['movieNm'],
                            item['openDt'][:4]])
      selected_movie_list.append(item['movieNm'])
  
print(movie_cd_list)
print(selected_movie_list)

[['20204117', '모가디슈', '2021'], ['20184571', '천문: 하늘에 묻는다', '2019'], ['20167904', '라라랜드', '2016'], ['20216362', '크루엘라', '2021'], ['20191951', '블랙 위도우', '2021']]
['모가디슈', '천문: 하늘에 묻는다', '라라랜드', '크루엘라', '블랙 위도우']


In [ ]:
import pandas as pd

movie_cdlist_df = pd.DataFrame(movie_cd_list)
movie_cdlist_df.columns = ['movieCd', 'movieNm','openYr']
movie_cdlist_df

,movieCd,movieNm,openYr
0,20204117,모가디슈,2021
1,20184571,천문: 하늘에 묻는다,2019
2,20167904,라라랜드,2016
3,20216362,크루엘라,2021
4,20191951,블랙 위도우,2021


영화코드로 배우추출

In [ ]:
movie_detail_list = []
movie_director_list = []
movie_cast_list = []    # 한 영화당 최대 10명의 배우를 추출하여 저장한 리스트
direct_list = []
movie_mainactor_list = []    # 한 영화당 최대 2명(주인공)의 배우를 추출하여 저장한 리스트
actor_list = []              # 5개 영화의 movie_cast_list을 모은 것
mainactor_list = []          # 5개 영화의 movie_mainactor_list을 모은 것
movie_actor_list = [[] for i in range(0,5)]
movie_direc_list = [[] for i in range(0,5)]
j=0

for movieCd in movie_cdlist_df['movieCd']:
  movie_director_list.clear()
  movie_cast_list.clear()
  movie_mainactor_list.clear()
  actor_list_url = 'https://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieInfo.json?key='+yourcode+'&movieCd='+movieCd
  res = requests.get(actor_list_url)
  text = res.text
  d = json.loads(text)

  for direc in d['movieInfoResult']['movieInfo']['directors']:
    movie_director_list.append(direc['peopleNm'])
  direct_list+=movie_director_list

  for cast in d['movieInfoResult']['movieInfo']['actors']:
    if len(movie_mainactor_list) < 2:
      movie_mainactor_list.append(cast['peopleNm'])
    if len(movie_cast_list) <= 9:
      movie_cast_list.append(cast['peopleNm'])
  #print(movie_cast_list)
  actor_list += movie_cast_list
  mainactor_list += movie_mainactor_list

  j+=1

print(direct_list)
print(actor_list)
print(mainactor_list)

['류승완', '허진호', '데이미언 셔젤', '크레이그 질레스피', '케이트 쇼트랜드']
['김윤석', '조인성', '허준호', '구교환', '김소진', '정만식', '김재화', '박경혜', '박명신', '한철우', '최민식', '한석규', '신구', '김홍파', '허준호', '김태우', '김원해', '임원희', '오광록', '박성훈', '엠마 스톤', '라이언 고슬링', '핀 위트록', 'J.K. 시몬스', '로즈마리 드윗', '엠마 스톤', '엠마 톰슨', '마크 스트롱', '스칼렛 요한슨', '플로렌스 퓨', '레이첼 와이즈', '데이빗 하버']
['김윤석', '조인성', '최민식', '한석규', '엠마 스톤', '라이언 고슬링', '엠마 스톤', '엠마 톰슨', '스칼렛 요한슨', '플로렌스 퓨']


배우 등장 횟수 세기

In [ ]:
actor_dict = dict()
for i in actor_list:
  if i not in actor_dict:
    actor_dict[i]=1
  else:
    actor_dict[i] = actor_dict[i]+1

actor_dict

{'J.K. 시몬스': 1,
 '구교환': 1,
 '김소진': 1,
 '김원해': 1,
 '김윤석': 1,
 '김재화': 1,
 '김태우': 1,
 '김홍파': 1,
 '데이빗 하버': 1,
 '라이언 고슬링': 1,
 '레이첼 와이즈': 1,
 '로즈마리 드윗': 1,
 '마크 스트롱': 1,
 '박경혜': 1,
 '박명신': 1,
 '박성훈': 1,
 '스칼렛 요한슨': 1,
 '신구': 1,
 '엠마 스톤': 2,
 '엠마 톰슨': 1,
 '오광록': 1,
 '임원희': 1,
 '정만식': 1,
 '조인성': 1,
 '최민식': 1,
 '플로렌스 퓨': 1,
 '핀 위트록': 1,
 '한석규': 1,
 '한철우': 1,
 '허준호': 2}

주연배우(출연진리스트 맨 앞 2사람) 등장 횟수

In [ ]:
mainactor_dict = dict()
for i in mainactor_list:
  if i not in mainactor_dict:
    mainactor_dict[i]=1
  else:
    mainactor_dict[i] = mainactor_dict[i]+1

mainactor_dict

{'김윤석': 1,
 '라이언 고슬링': 1,
 '스칼렛 요한슨': 1,
 '엠마 스톤': 2,
 '엠마 톰슨': 1,
 '조인성': 1,
 '최민식': 1,
 '플로렌스 퓨': 1,
 '한석규': 1}

추천해줄 배우 5명 선별
* 2번 이상 등장한 배우 우선 선발
* 빈자리는 주연배우 리스트에서 랜덤으로 고름

In [ ]:
import random

prefer_actor_list = []
for key, value in actor_dict.items():
  if value > 1:
    prefer_actor_list.append(key)
if len(prefer_actor_list)<5:
  while True:
    actor = random.choice(mainactor_list)
    if actor not in prefer_actor_list:
      prefer_actor_list.append(actor)
    if len(prefer_actor_list)==5:
      break
  
prefer_actor_list

['허준호', '엠마 스톤', '스칼렛 요한슨', '엠마 톰슨', '조인성']

배우 코드 추출

In [ ]:
actor_code = []
filmo_list = []
final_filmo_list = []

for prefer_actor in prefer_actor_list:
  actor_list_url = 'http://kobis.or.kr/kobisopenapi/webservice/rest/people/searchPeopleList.json?key='+yourcode+'+'&peopleNm='+prefer_actor
  res = requests.get(actor_list_url)
  text = res.text
  d = json.loads(text)

  for actor in d['peopleListResult']['peopleList']:
    final_filmo_list.clear()
    if actor['repRoleNm'] == '배우' and actor['peopleNmEn']!= "":
      filmo_list = actor['filmoNames'].split("|")
      for filmo in filmo_list:
        if filmo not in selected_movie_list:
          final_filmo_list.append(filmo)
      actor_code.append([actor['peopleCd'], actor['peopleNm'],actor['peopleNmEn'],
                         actor['repRoleNm'], final_filmo_list[:5]])

In [ ]:
print(selected_movie_list)
print(filmo_list)

['모가디슈', '천문: 하늘에 묻는다', '라라랜드', '크루엘라', '블랙 위도우']
['모가디슈', '안시성', '더 킹', '쌍화점', '비열한 거리', '남남북녀', '클래식', '마들렌', '화장실, 어디에요?', '밀수', '조인성을 좋아하세요']


In [ ]:
actor_code_df = pd.DataFrame(actor_code)
actor_code_df.columns = ['peopleCd','peopleNm','peopleNmEn','repRoleNm','filmoNames']
actor_code_df

,peopleCd,peopleNm,peopleNmEn,repRoleNm,filmoNames
0,10088453,허준호,HUH Joon-ho,배우,"[노량: 죽음의 바다, 결백, 퍼펙트맨, 국가부도의 날, 인랑]"
1,10049633,엠마 스톤,Emma Stone,배우,"[크루즈 패밀리: 뉴 에이지, 좀비랜드: 더블 탭, 더 페이버릿: 여왕의 여자, 빌..."
2,10038163,스칼렛 요한슨,Scarlett Johansson,배우,"[씽2게더, 조조 래빗, 결혼 이야기, 어벤져스: 엔드게임, 개들의 섬]"
3,10049649,엠마 톰슨,Emma Thompson,배우,"[닥터 두리틀, 라스트 크리스마스, 잃어버린 세계를 찾아서, 맨 인 블랙: 인터내셔..."
4,10066899,조인성,ZO In-sung,배우,"[안시성, 더 킹, 쌍화점, 비열한 거리, 남남북녀]"
